In [124]:
import pandas as pd
import seaborn as sns
import numpy as np


In [125]:
movies = "../data/movie_lens_big/movies.csv"
ratings = "../data/movie_lens_big/ratings.csv"

In [126]:
df_movies = pd.read_csv(movies, dtype={"title":"str", "genres":"str"})
df_ratings = pd.read_csv(ratings)

In [127]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [128]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


In [129]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [130]:
df_movies["genres"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 58098 entries, 0 to 58097
Series name: genres
Non-Null Count  Dtype 
--------------  ----- 
58098 non-null  object
dtypes: object(1)
memory usage: 454.0+ KB


In [131]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [132]:
# dropping userId and timestamp from the dataframe
df_ratings = df_ratings.drop(["userId", "timestamp"], axis=1)
df_ratings.head()

,movieId,rating
0,307,3.5
1,481,3.5
2,1091,1.5
3,1257,4.5
4,1449,4.5


In [133]:
df_movies["year"] = df_movies.title.str.extract("(\d+)").replace(np.nan, "0").astype("int64") # extracting the year from title and creating new column 
df_movies["title"] = df_movies["title"].str.replace("\d+", '').str.replace(r"\(.*\)","") # This removes the brackets



df_movies.head()

C:\Users\Sam Glass ITHS\AppData\Local\Temp\ipykernel_12920\2172042238.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_movies["title"] = df_movies["title"].str.replace("\d+", '').str.replace(r"\(.*\)","") # This removes the brackets


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [134]:
dummies = df_movies["genres"].str.get_dummies(sep="|") # This seperates all the genres into columns with 1 meaning true
df_movies_genre_split = pd.concat([df_movies, dummies], axis=1)

df_movies_genre_split.head()

,movieId,title,genres,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,Adventure|Children|Fantasy,1995,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,Comedy|Romance,1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,Comedy,1995,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [135]:
# making a mean average rating of the movies from movie id and rating
average_rating = df_ratings.groupby("movieId")["rating"].mean().reset_index().round(decimals=1)
average_rating.rename(columns={"rating":"av_rating"}, inplace=True)
average_rating.head()

,movieId,av_rating
0,1,3.9
1,2,3.2
2,3,3.2
3,4,2.9
4,5,3.1


In [136]:
amount_ratings = df_ratings[["movieId", "rating"]].value_counts(subset=["movieId"]).to_frame(name="Total ratings").reset_index()
amount_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53889 entries, 0 to 53888
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   movieId        53889 non-null  int64
 1   Total ratings  53889 non-null  int64
dtypes: int64(2)
memory usage: 842.1 KB


In [172]:
# while looking at the years of the data you can see that there are some that are not real years
# I have decided that I will only have movies between 1939 and 2022 as 1939 was the real revolution of color films
# such as The wizard of Oz. 
df_complete_movie = df_movies.merge(average_rating, on="movieId").merge(amount_ratings, on="movieId")
df_complete_movie = df_complete_movie.sort_values(by=["year"], ascending=True)
df_complete_movie.head(2900)

,movieId,title,genres,year,av_rating,Total ratings
28969,129651,The Third Reich: The Rise & Fall,(no genres listed),0,3.0,1
41268,163018,A Lovasíjász,Adventure|Documentary,0,3.5,1
40348,160579,Friends and Romans,(no genres listed),0,1.8,2
53200,192097,.mm,Drama,0,3.2,2
49934,183837,The Favourite,Drama,0,3.0,3
...,...,...,...,...,...,...
46832,176225,The Living Playing Cards,Fantasy,1905,3.0,1
27916,125996,The Black Devil,Comedy|Fantasy,1905,2.8,9
47057,176743,King of Dollars,Fantasy,1905,3.0,1
39844,159149,Rescued by Rover,Drama,1905,3.0,4


In [175]:
df_complete_movie = df_complete_movie.loc[df_complete_movie["year"] <=2022]
df_complete_movie = df_complete_movie.loc[df_complete_movie["year"] >=1939]
df_complete_movie.head()

,movieId,title,genres,year,av_rating,Total ratings
18933,93775,Seitsemän veljestä,Drama,1939,3.1,4
39530,158384,Buck Rogers,Sci-Fi,1939,3.5,2
34664,144740,Sighs of Spain,(no genres listed),1939,2.5,1
42709,166737,Fisherman's Wharf,Adventure|Drama,1939,3.0,1
8432,25858,"Story of the Late Chrysanthemums, The",Drama,1939,3.8,20
